In [8]:
import pandas as pd
import json
from pandas import Series,DataFrame
import re
import datetime
from bs4 import BeautifulSoup
import requests
#寻找小区对应的区域和建造年份
def get_data(x):
    try:
        html=requests.get('https://dl.lianjia.com/xiaoqu/rs'+x)
        soup=BeautifulSoup(html.text,'lxml')
        s=soup.select('.info .title a')
        for i in range(0,len(s)):
            if s[i].string==x:
                district=soup.select('.info .positionInfo .district')[i].string
                bizcircle=soup.select('.info .positionInfo .bizcircle')[i].string
                temp=soup.select('.info .positionInfo')[i].get_text()
                nianfen=re.search('\d+',temp).group()
                return x+' '+district+' '+bizcircle+' '+ str(nianfen)
        else:
            return None
    except IndexError as e:
        print(x)

pd.set_option('display.max_rows',None)
# #设置显示全部列，不省略
pd.set_option('display.max_columns',None)

item=[]
for line in open('data/lianjia.json','r',encoding='utf-8'):
    item.append(json.loads(line))
    
df=DataFrame(item)
df.drop_duplicates(inplace=True)

 

In [9]:
getarea=lambda x: float(re.search(r"\d+(\.\d+)?",x).group())
df['area']=df['area'].apply(getarea)
tofloat=lambda x:float(x)
df['unitPrice']=df['unitPrice'].apply(tofloat)
df['totalPrice']=df['totalPrice'].apply(tofloat)
df['dealDate']=pd.to_datetime(df['dealDate'])
df['dealCycle']=df['dealCycle'].map(lambda x:int(re.search('\d+',x).group()))
df['guaPai']=df['guaPai'].apply(getarea)

In [10]:
#找出降价幅度大的房子
f=df[((df['guaPai']-df['totalPrice'])/df['guaPai']>0.05) & (df['dealDate'].map(lambda x:x.year)==2021)].sort_values(by=['title','dealDate'])
f['chazhi']=round((f['guaPai']-f['totalPrice'])/f['guaPai'],2)

In [11]:
#找出各个销售的销售量，目前看工人村单个销售卖的还真多
s=df.groupby(['seller'])[['seller']].count()
s.index.name='num'
s.sort_values(by=['seller'],ascending=False)
s.head()

,seller
num,
SSC业务运营中心,9
丁一航,1
丁丽君,1
丁伟洧,1
丁余艳,14


In [34]:
#找出各小区的销售数量，先忽略时间
# xiaoqu=df.groupby(['title'])[['title']].count()
xiaoqu=df['title'].groupby([df['title'],df['dealDate'].apply(lambda x:x.year)]).count()
xiaoqu.index.name='num'
xiaoqu['第五郡五号地']


dealDate
2012     1
2016    12
2017    17
2018     9
2019    18
2020    12
2021    23
Name: title, dtype: int64

In [40]:
#找出某个小区的销售情况
print(get_data('第五郡五号地'))
df_wu=df[df['title'].apply(lambda x: '第五郡' in x and '公寓' not in x)]
df_wu=df_wu.sort_values(by='dealDate')
df_wu=df_wu.reset_index(drop=True)
func=lambda x:str((x.year)*100+(x.month))[0:4]+'-'+str((x.year)*100+(x.month))[4:]
df_wu['month']=df_wu['dealDate'].map(func)
df_wu['title'].groupby(df_wu['month']).count()
# df[df['title']=='金海花园西园'][['title']].count()

第五郡五号地 甘井子 第五郡 2006


month
2012-08     1
2016-03     1
2016-05     1
2016-07     2
2016-08     3
2016-11     3
2016-12     2
2017-01     2
2017-02     1
2017-03     5
2017-04     4
2017-05     4
2017-06     1
2018-04     2
2018-07     1
2018-09     3
2018-10     2
2018-11     1
2019-01     1
2019-03     4
2019-04     4
2019-05     3
2019-06     2
2019-07     3
2019-09     1
2020-07     1
2020-09     3
2020-10     1
2020-11     3
2020-12     4
2021-01     1
2021-02     6
2021-03     5
2021-04    10
2021-05     1
Name: title, dtype: int64

In [41]:
#找出小区每年均价，按月标本太少，不太准确
df_wu['unitPrice'].groupby(df_wu['dealDate'].map(lambda x:x.year)).describe()

,count,mean,std,min,25%,50%,75%,max
dealDate,,,,,,,,
2012,1.0,17582.000000,NaN,17582.0,17582.00,17582.0,17582.00,17582.0
2016,12.0,16827.000000,1485.846193,13386.0,16356.50,17206.0,17966.00,18315.0
2017,17.0,18020.176471,2415.059193,14289.0,16246.00,18097.0,19008.00,22142.0
2018,9.0,24406.444444,3349.553706,19848.0,22939.00,24473.0,26068.00,30461.0
2019,18.0,25171.166667,2292.345402,22182.0,23884.50,24755.5,25285.25,31291.0
2020,12.0,27258.583333,4602.213962,19075.0,25399.25,26602.0,27424.25,38650.0
2021,23.0,26557.000000,2048.363538,22107.0,25326.00,26911.0,28163.50,29239.0


In [42]:
#找出小区的成交周期
df_wu['dealCycle'].groupby(df_wu['dealDate'].apply(lambda x:x.year)).describe()

,count,mean,std,min,25%,50%,75%,max
dealDate,,,,,,,,
2012,1.0,1.000000,NaN,1.0,1.00,1.0,1.00,1.0
2016,12.0,148.916667,87.619486,37.0,87.75,133.0,181.75,309.0
2017,17.0,358.000000,146.302768,71.0,258.00,375.0,455.00,598.0
2018,9.0,98.000000,59.396970,6.0,55.00,122.0,140.00,177.0
2019,18.0,152.833333,159.870260,2.0,30.50,111.0,243.75,622.0
2020,12.0,180.666667,192.607624,3.0,72.50,128.0,197.25,724.0
2021,23.0,190.782609,188.907134,1.0,47.00,141.0,259.00,689.0


In [20]:
#找出房子里只有一天成交周期的房子
df[(df['dealCycle'].map(lambda x:x==1)) & (df['dealDate'].apply(lambda x:x.year)==2021)]

,title,room,area,dealDate,totalPrice,unitPrice,dealCycle,guaPai,seller
76,中庚香海金鼎回迁楼,2室1厅,68.00,2021-04-28,36.0,5221.0,1,36.0,于丽华
77,中庚香海金鼎回迁楼,1室1厅,61.42,2021-03-04,21.0,3420.0,1,21.5,何新玉
961,高洁巷,2室1厅,51.10,2021-04-20,62.0,12134.0,1,62.5,刘彦秀
1566,MINI印象,1室0厅,50.80,2021-03-27,84.0,16536.0,1,86.0,周浩
2236,中海凤凰熙岸,2室2厅,73.80,2021-02-21,109.0,14770.0,1,110.0,付霞
2560,新居佳园,1室1厅,50.24,2021-04-18,77.0,15227.0,1,79.0,刘妍研
3819,泡崖四区,2室1厅,62.48,2021-05-12,86.0,13685.0,1,86.0,佟秀杰
3825,水仙花园(西岗),2室2厅,125.57,2021-04-26,463.0,36872.0,1,465.0,周卫祯一
3846,泡崖四区,2室1厅,62.82,2021-03-05,78.0,12369.0,1,79.5,盛香爽
4160,亿诚御景湾三期,2室1厅,77.90,2021-01-21,55.0,6997.0,1,55.0,杜洪涛


In [ ]:
#备用功能        
# list=[]
# for i in df['title'].drop_duplicates():
#     g=get_data(i)
#     if g is not None:
#         list.append(g)

# %run main.py
#df.to_json(path_or_buf='data/test.json',orient='index',force_ascii=False)
#检查area数据
for i in range(0,len(df)):
    result=df.iloc[i]
    if re.search(r'\d+(\.\d+)?(平米)',result[2]) is None:
        print(df.iloc[i])
        df.drop([i])
#检查挂牌数据
for i in range(0,len(df)):
    result=df.iloc[i]
    if re.search('挂牌*',result[7]) is None:
        print(str(i))
        # df.drop([i])